Import Necessary Libraries:

In [ ]:
pip install shap scikit-learn pandas numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split




Load Your Data:
Load and prepare your data as described earlier.

Train Models and Generate SHAP Values:
Train a model for each target column and compute SHAP values.

In [ ]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import os
from PIL import Image

# Load your dataset from Excel
file_path = 'cleaned_imputed_data.xlsx'
data = pd.read_excel(file_path)

# Define feature columns and target columns
feature_columns = [
    'Carbon content (wt%)',
    'Hydrogen content (wt%)',
    'Nitrogen content (wt%)',
    'Oxygen content (wt%)',
    'Sulfur content (wt%)',
    'Volatile matter (wt%)',
    'Fixed carbon (wt%)',
    'Ash content (wt%)',
    'Reaction temperature (°C)',
    'Microwave power (W)',
    'Reaction time (min)',
    'Microwave absorber percentage (%)',
    'Dielectric constant of absorber (ε′)',
    'Dielectric loss factor of absorber (ε“)'
]

target_columns = [
    'Bio-oil yield (%)',
    'Syngas yield (%)',
    'Syngas composition (H₂, mol%)',
    'Syngas composition (CH₄, mol%)',
    'Syngas composition (CO₂, mol%)',
    'Syngas composition (CO, mol%)',
    'Biochar yield (%)',
    'Biochar calorific value (MJ/kg)',
    'Biochar H/C ratio (-)',
    'Biochar H/N ratio (-)',
    'Biochar O/C ratio (-)'
]

# Extract features and targets
X = data[feature_columns]
y = data[target_columns]

# Directory to save plots
plot_dir = 'shap_plots/'
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

def sanitize_filename(filename):
    return ''.join(c for c in filename if c.isalnum() or c in (' ', '_')).rstrip()

# Generate and save SHAP summary plots
for target_column in target_columns:
    print(f"Analyzing target: {target_column}")

    # Extract the target values
    y_target = y[target_column]

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.2, random_state=42)

    # Train a Random Forest regressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Create a SHAP explainer
    explainer = shap.TreeExplainer(model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X_test)

    # Save the summary plot to a file
    try:
        plt.figure(figsize=(10, 5))
        shap.summary_plot(shap_values, X_test, show=False)
        plt.title(f"SHAP Summary Plot for {target_column}")

        # Sanitize the filename
        safe_target_column = sanitize_filename(target_column)
        filename = os.path.join(plot_dir, f'shap_summary_{safe_target_column}.png')
        plt.savefig(filename)
        plt.close()
    except Exception as e:
        print(f"Failed to save plot for {target_column}: {e}")

# Combine saved SHAP plot images into one large image
plot_files = sorted([os.path.join(plot_dir, f) for f in os.listdir(plot_dir) if f.endswith('.png')])

# Open images
images = [Image.open(f) for f in plot_files]

# Assuming all images are the same size, get dimensions
width, height = images[0].size

# Create a new image with a height that can fit all plots
total_height = height * len(images)
combined_image = Image.new('RGB', (width, total_height))

# Paste each image into the combined image
y_offset = 0
for img in images:
    combined_image.paste(img, (0, y_offset))
    y_offset += height

# Save and show the combined image
combined_image_path = 'combined_shap_plots.png'
combined_image.save(combined_image_path)

# Display the combined image
combined_image.show()


Analyzing target: Bio-oil yield (%)
Analyzing target: Syngas yield (%)
Analyzing target: Syngas composition (H₂, mol%)
Analyzing target: Syngas composition (CH₄, mol%)
Analyzing target: Syngas composition (CO₂, mol%)
Analyzing target: Syngas composition (CO, mol%)
Analyzing target: Biochar yield (%)
Analyzing target: Biochar calorific value (MJ/kg)
Analyzing target: Biochar H/C ratio (-)
Analyzing target: Biochar H/N ratio (-)
Analyzing target: Biochar O/C ratio (-)
